# Code

## Imports and packages required

In [1]:
import numpy as np
import pandas as pd

## Preprocessing

### 1. Reading the CSV File

In [2]:
df = pd.read_csv('../data/online_retail_listing.csv', on_bad_lines='skip',delimiter=';', decimal = ',')
df.head(3)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,1.12.2009 07:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,1.12.2009 07:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,1.12.2009 07:45,6.75,13085.0,United Kingdom


### 2. InvoiceDate column type conversion from String to DateTime

In [3]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format = "%d.%m.%Y %H:%M")

### 3. Customer ID column type conversion from Float to Int

In [4]:
df['Customer ID'] = df['Customer ID'].astype('Int64')

### 4. Quantity and price - Filter out bad values

In [5]:
df = df[df['Quantity']>=0]
df = df[df['Price']>=0]

### 5. Processing the Description

1. Convert all descriptions to uppercase
2. Find `StockCode`s with multiple descriptions (Lengthy, Short, NaN, etc.) and keep the longest string as the detailed description
3. Filter out transactions with no descriptions (66 found)
4. Strip the string to remove any leading and trailing whitespaces

In [6]:
#1
df["Description"] = df["Description"].str.upper()

#2
grps = df[["StockCode", "Description"]].drop_duplicates().groupby("StockCode")
desc_replacer = dict()
for grp in grps:
    if len(grp[1])>1:
        desc_replacer[grp[0]] = grp[1]["Description"][grp[1]["Description"].map(str).agg(len).idxmax()]

df["Description"] = df.apply(
        axis=1, 
        func=lambda x: desc_replacer[x["StockCode"]] if x["StockCode"] in desc_replacer.keys() else x["Description"]
    )

#3
df = df[df['Description'].notna()]

#4
df["Description"] = df["Description"].map(lambda x: str(x).strip())

### 6. Processing the Customer IDs

In the dataset, we know that there is no such entry where the same invoice is shared between two customers.

Let us check if the same invoice has a customer ID in some entries, and NA in other

In [7]:
df["Customer ID"].isna().sum()

232427

In [8]:
empty_CID_Orders = df[df["Customer ID"].isna()]["Invoice"].drop_duplicates().values.tolist()
display(df[df["Invoice"].map(lambda x: x in empty_CID_Orders)]["Customer ID"].notna().sum())

0

Since there are no invoices where we can recover `Customer ID` from another entry, we can give placeholder IDs to Customers with ID `NA`, assuming each invoice corresponds to a new customer.

In [9]:
max_existing_id = df["Customer ID"].max()
invoice_cust_dict = dict()

for key in empty_CID_Orders:
    max_existing_id += 1
    invoice_cust_dict[key] = max_existing_id

df["Valid_Customer"] = df["Customer ID"].notna().map(lambda x: int(x))
df["Customer ID"] = df.apply(
        axis=1, 
        func=lambda x: invoice_cust_dict[x["Invoice"]] if x["Invoice"] in invoice_cust_dict.keys() else x["Customer ID"]
    )

### 7. Drop duplicate entries of the same transaction

In [11]:
df.drop_duplicates(inplace=True)

### 8. Check for invalid data in the DataFrame

In [12]:
df.isna().sum()

Invoice           0
StockCode         0
Description       0
Quantity          0
InvoiceDate       0
Price             0
Customer ID       0
Country           0
Valid_Customer    0
dtype: int64

### 9. Save the cleaned DataFrame for further analysis

In [13]:
df.to_csv('../data/online_retail_listing_cleaned.csv', index=False)

## Exploratory Data Analysis

In [ ]:
#!pip install pandas-profiling

In [ ]:
import pandas_profiling
#Generating PandasProfiling Report
report = pandas_profiling.ProfileReport(df)

In [ ]:
report

In [ ]:
df.drop_duplicates()

In [ ]:
# skewness along the index axis
df.skew(axis = 0, skipna = True)# highly skewed price, indicating there might be a few low priced items 

In [ ]:
df['Country'].unique()


## Market Basket Analysis

Unstack the df into basket with invoice and product descriptions

In [ ]:
basket = df.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('Invoice')

In [ ]:
basket 

Encode 0.0 to 0 and 1.0 to 1

In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
    
basket_encode_plus = basket.applymap(encode_units)
basket_encode_plus

Filter out Invoices with atleast 2 products 

In [ ]:
basket_filter_plus = basket_encode_plus[(basket_encode_plus > 0).sum(axis=1) >= 2]
basket_filter_plus

In [ ]:
#!pip install mlxtend

In [ ]:
from mlxtend.frequent_patterns import apriori
frequent_itemsets_plus = apriori(basket_filter_plus, min_support=0.03, 
                                 use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)

frequent_itemsets_plus['length'] = frequent_itemsets_plus['itemsets'].apply(lambda x: len(x))

frequent_itemsets_plus

In [ ]:
from mlxtend.frequent_patterns import association_rules

In [ ]:
association_rules(frequent_itemsets_plus, metric='lift', 
                  min_threshold=1).sort_values('lift', ascending=False).reset_index(drop=True)

# Customer Segmentation